# Question 1

## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## Creating the Dataframe

In [2]:
#Read Table
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
df = pd.read_html(url)[0]
df.shape
#df.head(15)

(288, 3)

## Cleanup Dataframe

In [3]:
#Drop 'Not Assigned Boroughs'
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#Rename Columns to Match Assigment Dataframe Example
df=df.rename(columns = {'Postcode':'PostalCode'})
df=df.rename(columns = {'Neighbourhood':'Neighborhood'})
df.shape
#df.head()

(211, 3)

In [4]:
df1 = df.reset_index()
#df1.shape
#df1.info

## Combine Multiple Neighborhoods on the Same Postal Code

In [5]:
df2 = df1.groupby('PostalCode').agg(lambda x: ', '.join(x))
df2.head(10)

,Borough,Neighborhood
PostalCode,,
M1B,"Scarborough, Scarborough","Rouge, Malvern"
M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union"
M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough, Scarborough, Scarborough","East Birchmount Park, Ionview, Kennedy Park"
M1L,"Scarborough, Scarborough, Scarborough","Clairlea, Golden Mile, Oakridge"
M1M,"Scarborough, Scarborough, Scarborough","Cliffcrest, Cliffside, Scarborough Village West"


## If a cell has a borough but a Not assigned neighborhood, then change the neighborhood to be the same as the borough

In [6]:
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough']
#df2

## Delete Multiple Boroughs Per Postal Code

In [7]:
df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [8]:
df2.sort_values(by=['PostalCode'], inplace=True)
df2.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [9]:
df2.shape

(103, 2)